### **Imports**

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

from robolab_turtlebot import Turtlebot
from TurtleControllers import TurtlebotController
from TurtleVision import TurtlebotVision
from TurtleMap import TurtlebotMap
from TurtleICP import TurtlebotICP
import TurtleUtils
import time

### **Initialization**

In [ ]:
turtle = Turtlebot(rgb = True, pc = True)
turtle_controller = TurtlebotController(turtle, rate = 40)
turtle_vision = TurtlebotVision(turtle)
turtle_map_yellow = TurtlebotMap()  # Map for yellow points in images
turtle_map_purple = TurtlebotMap()  # Map for purple points in images
turtle_icp = TurtlebotICP()

### **Scanning**

In [ ]:
def scan(r2p = False):
    ANGLE_STEP = math.radians(30)
    for angle in range(12):
        # Scan environment
        yellow_points = turtle_vision.sample_garage(color = "yellow", r2p = r2p)
        purple_points = turtle_vision.sample_garage(step_size = 3, 
                                color = "purple", minimal_size = 8, r2p = r2p)
        
        # Current odometry
        odom = turtle_controller.get_odometry()

        # Add points to maps
        turtle_map_yellow.add_points(yellow_points, odom)
        turtle_map_purple.add_points(purple_points, odom)

        # Rotate to next position
        turtle_controller.rotate(ANGLE_STEP, relative = True)
        time.sleep(1.0)

### **Fit garage model**

In [ ]:
def task2():
    # Scan environment
    scan()

    # Task1 if there are some purple clusters
    use_task1 = turtle_map_purple.cluster_count >= 2

    if use_task1:
        print("Use task1")
        return
    
    turtle_map_purple.reset()

    # Fit garage model using ICP
    yellow_downsampled = turtle_map_yellow.points_downsampled
    opt = turtle_icp.fit(yellow_downsampled)
    garage = opt.garage

    # For now assume the garage fit is perfect
    pt, idx, should_scan = garage.closest_waypoint(turtle_controller.get_odometry())


    while idx != 0:
        # Move to closest waypoint
        turtle_controller.move_to(pt, relative = False)

        if should_scan:
            task2()
            return
        
        pt, idx, should_scan = garage.closest_waypoint(idx)

    # Should be near the entrance
    task2()